In [1]:
# https://www.nih.gov/news-events/news-releases/nih-clinical-center-provides-one-largest-publicly-available-chest-x-ray-datasets-scientific-community

import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import os
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms as tf
from os import walk
import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from matplotlib import image
from torchvision import transforms
# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [2]:
train_df = pd.read_csv('train_df2.csv')
test_df = pd.read_csv('test_df2.csv')

In [3]:
train_df.sample(10)

,Finding Label,Image Index,h,w,x,y
12298,1,00003847_008.png,2048.0,2500.0,0.168,0.168
35378,0,00010949_002.png,2048.0,2500.0,0.168,0.168
57033,1,00017728_000.png,2048.0,2500.0,0.168,0.168
15992,1,00005138_004.png,3056.0,2544.0,0.139,0.139
6690,1,00002166_000.png,2991.0,2598.0,0.143,0.143
40719,13,00012585_005.png,2048.0,2500.0,0.168,0.168
19455,1,00006147_000.png,2500.0,2048.0,0.171,0.171
53538,1,00016636_000.png,2048.0,2500.0,0.168,0.168
32545,1,00010270_001.png,2833.0,2738.0,0.143,0.143
85745,13,00030038_009.png,2544.0,3056.0,0.139,0.139


In [4]:
# (train_df['Image Index'] == '00030181_001.png').value_counts()
# 00028247_001.png

In [5]:
import os
import platform
my_path = "../Datasets/Lungs_Dataset/Xray" if platform.system() == 'Windows' else "datasets/data/images"
filename_list = []
for root, dirs, files in os.walk(my_path, topdown=True):
    for name in files:
        filename_list.append(name)
#         with Image.open(os.path.join("datasets/data/images", name)) as f:
#             print(len(f.size))

In [6]:
train_df['Image Index'].isin(filename_list).value_counts()

True     82995
False     3529
Name: Image Index, dtype: int64

In [7]:
train_df = train_df[train_df['Image Index'].isin(filename_list)]

In [8]:
train_df['Image Index'].isin(filename_list).value_counts()

True    82995
Name: Image Index, dtype: int64

In [9]:
train_df['Image Index'].isna().value_counts()

False    82995
Name: Image Index, dtype: int64

In [10]:
test_df = test_df[test_df['Image Index'].isin(filename_list)]

In [11]:
test_df.sample(10)

,Finding Label,Image Index,h,w,x,y
11713,1,00014664_006.png,2048.0,2500.0,0.168000,0.168000
9559,2,00013278_000.png,2048.0,2500.0,0.168000,0.168000
19303,1,00022815_063.png,2544.0,3056.0,0.139000,0.139000
1292,1,00001836_027.png,2048.0,2500.0,0.168000,0.168000
18769,8,00022098_009.png,2020.0,2021.0,0.194311,0.194311
17889,1,00020945_057.png,2544.0,3056.0,0.139000,0.139000
20667,3,00026221_017.png,2544.0,3056.0,0.139000,0.139000
13571,1,00016508_029.png,2048.0,2500.0,0.168000,0.168000
5754,9,00009403_004.png,2048.0,2500.0,0.168000,0.168000
15322,12,00018233_023.png,2544.0,3056.0,0.139000,0.139000


In [12]:
class RescaleImage(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
            
        The samples coming into this class will have its images reduced assuming
        the input is a h, w, c numpy array
    """

    def __init__(self, output_size):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        h, w = image.size

        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size

        new_h, new_w = int(new_h), int(new_w)

        img = image.resize((new_h, new_w))

        return {'image': img, 'label': label}

In [13]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        image = np.array(image)
        if len(image.shape) > 2:
            image = image[:,:,0]

        # numpy image: H x W x C
        # torch image: C X H X W
        return {'image': torch.FloatTensor(image).unsqueeze(0),
#         return {'image': torch.from_numpy(image),                
                'label': torch.from_numpy(label)}

In [14]:
#  print([Image.open('../Datasets/Lungs_Dataset/Xray/'+train_df['Image Index'][each_import]).size for each_import in range(100)])

In [15]:
#Note this will return an Image object, of h and w, and its corresponding label
class CovidLungsDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, dataframe, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataframe = dataframe
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.dataframe['Image Index'][idx])
        my_image = Image.open(img_name)        
        if len(my_image.size) > 2:
            assert len(my_image.size) > 2
        row = self.dataframe.iloc[idx]
        label = np.array([row['Finding Label']])
        sample = {'image': my_image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [16]:
my_train_set = CovidLungsDataset(train_df, my_path, transform=transforms.Compose([
        RescaleImage(200),
        ToTensor()
]))

In [17]:
'''The original data coming out of the Dataset is a dictionary, 
the image is an Image object with the corresponding label.
The size of all of these images are 1024 by 1024.
The RescaleImage will convert the Image object to an Image object with 200x200 in size, 
leaving the label alone.
The ToTensor will convert the Image object with 200x200 to tensors of 1x200x200'''
from random import randint
my_image = my_train_set.__getitem__(randint(0,256))['image']
#imshow will work if it's h,w,c or h,w
#torch is c,h,w
#line below can be used if you don't use ToTensor
# plt.imshow(my_image)
type(my_image), my_image.shape

(torch.Tensor, torch.Size([1, 200, 200]))

In [18]:
# plt.imshow(my_image.squeeze(0));

In [19]:
import platform
batch_loader_params = {
    "batch_size": 5,
    "shuffle": True,
    "num_workers": 0 if platform.system() == 'Windows' else 2
}
dataloader = DataLoader(my_train_set, **batch_loader_params)

In [20]:
# iter(dataloader).next()
for i, each in enumerate(dataloader):
    print(each['image'].shape, each['label'].shape)

torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size([5, 1])
torch.Size([5, 1, 200, 200]) torch.Size(

KeyboardInterrupt: 

In [70]:
# batch_samples = iter(dataloader)
# samples = batch_samples.next()
# datset_batch = torchvision.utils.make_grid(samples['image'])

In [71]:
# plt.figure(figsize=(20,10))
# for index, each in enumerate(datset_batch):
#     plt.imshow(each.squeeze(0))

In [21]:
# 1x200x200

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)  # 28x28x4 with batch of 6, 18816
#         self.conv2 = nn.Conv2d(20, 50, 5, 1)
#         self.fc1 = nn.Linear(4*4*50, 60)
#         self.fc2 = nn.Linear(60, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))  # 24x24x20 with at batch of 6
        return x

In [22]:
net = Net()

In [23]:
def spatial_size(input_size: int, kernel_size: int, stride: int = 1, padding: int = 0):
    # https://cs231n.github.io/convolutional-networks/
    spatial_size = (input_size - kernel_size + 2 * padding)/stride + 1
    assert spatial_size % 1 == 0
    assert spatial_size > 0
    return int(spatial_size)

In [24]:
print(spatial_size(200, 100))

101


In [25]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 100, 100, 1)  
        self.conv2 = nn.Conv2d(100, 50, 50, 1)
        self.fc1 = nn.Linear(4*4*50, 60)
        self.fc2 = nn.Linear(60, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
#         x = F.max_pool2d(x, 2, 2) 
#         x = F.relu(self.conv2(x)) 
#         x = F.max_pool2d(x, 2, 2) 
#         x = x.view(-1, 4*4*50)    
#         x = F.relu(self.fc1(x))   
#         x = self.fc2(x)           
        # # There's no activation at the final layer because of the criterion of CEL
        return x


net = Net()

In [27]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
#         inputs, labels = data['image'], data['label']
#         print(data['image'].shape)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(data['image'])
        loss = criterion(outputs, data['label'].squeeze())
#         loss.backward()
#         optimizer.step()

#         # print statistics
#         running_loss += loss.item()

#     print('[%d, %5d] loss: %.5f' %
#           (epoch + 1, i + 1, running_loss / (epoch*i+1)))

RuntimeError: only batches of spatial targets supported (3D tensors) but got targets of dimension: 1